<a href="https://colab.research.google.com/github/podschwadt/fhe_tutorial/blob/main/openfhe_ckks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenFHE

OpenFHE is a C++ library (https://github.com/openfheorg/openfhe-development). We will be using the official Python wrapper (https://github.com/openfheorg/openfhe-python). While the C++ version has more features, the Python wrapper provides all the features we need, is easier to use, and the naming conventions are identical to the C++ library, making it easy to switch over.

Currently, the OpenFHE Python wrapper does not provide a prebuilt package, and we need to build it ourselves. In the interest of time, I have built the package so that it works in Colab and uploaded it Github.

The cell below downloads my prebuilt package and installs it. If you are running this notebook outside of Colab you should **not** run this cell (it will probably fail anyway). Instead follow the installation instructions from the Github page (https://github.com/openfheorg/openfhe-python?tab=readme-ov-file#building-from-source)

In [ ]:
!wget https://github.com/podschwadt/fhe_tutorial/releases/download/1.0.0/openfhe.tar.gz
!tar xPf openfhe.tar.gz
!pip install openfhe-0.8.4-cp310-cp310-linux_x86_64.whl

Now that we installed the `openfhe` Python package we can import it.

In [ ]:
import openfhe

## Creating a Context

The most important class in OpenFHE is the `CryptoContext`. It represents
 the instantiation of a scheme and holds all the parameters, like ring dimension, etc. We can use the context to create keys, encode data, encrypt and decrypt data, and perform all sorts of operations on the ciphertexts.

 To create a Context, we first need to specify some parameters. At the very least, we need to select the scheme. To use the integer or real numbers schemes, we need to use the appropiate parameter class. The availabe classes are:

*   `CCParamsBFVRNS` for BFV
*   `CCParamsBGVRNS` for BFV
*   `CCParamsCKKSRNS` for CKKS

As you can see from the names, OpenFHE implements the RNS (residue number system) variants of the schemes for efficiency. The binary schemes do not have a dedicated parameter class. But more on that later.



We are going to create parameters to instantiate a CKKS context. For now, we won't set any specific parameters and will simply work with the default values.

In [ ]:
parameters = openfhe.CCParamsCKKSRNS()
# print the string represenation of the paramters
str(parameters)

We can see that parameters hold quite a lot of information. Many of these options will probably look unfamilar to you and infact many are inteded for experts, as they have security implications.

However, a few might look familar. Let's go over some of them.

*   `batchSize`: Is the number of slots in a plaintext/ciphertext that hold meanigful information. Here we have `0`, meaning that we don't want to create the context with a specific number of slots.
*   `multiplicativeDepth`: The number of multiplications before we can no longer correctly decrypt the ciphertext. Currently `1`
*    `scalingModSize`: The size (in bits) of the $q_i$ primes that we use to calcualte the ciphertext modulus $Q$. Needs to be less than 60. Currently `50`
*    `securityLevel`: The security level tells us how secure (in bits) the scheme instantiation will be. It is based on the recommendations in the HE standard (https://homomorphicencryption.org/wp-content/uploads/2018/11/HomomorphicEncryptionStandardv1.1.pdf). It is used to verify parameters such as ring dimension and ciphertext modulus or select appropriate ones if we don't specify any. Available **secure** values are `HEStd_128_classic`,`HEStd_192_classic`, and `HEStd_256_classic`. If we want to use less (or insecure) parameters for testing we can use `HEStd_NotSet`. Obviously, it is not recommended for any security-critical settings.
*    `ringDim` the dimension $N$ of the ring $Z_q[x]/(X^N+1)$. Currently set to `0` which means it will be determined later based on the other parameters.


Now, we can create a Context from our parameters. Once we have the context object we need to enable a few features. Here we are enabling `PKE` to allow for public key generation, `KEYSWITCH` which we need for relineraization (and later for rotations) and `LEVELEDSHE` for rescaling.

In [ ]:
context = openfhe.GenCryptoContext(parameters)
context.Enable(openfhe.PKESchemeFeature.PKE)
context.Enable(openfhe.PKESchemeFeature.KEYSWITCH)
context.Enable(openfhe.PKESchemeFeature.LEVELEDSHE)

Now that we have a context let's see what ring dimension was chosen.

In [ ]:
context.GetRingDimension()

## Encoding and Encryption

Using the context, we can encode a message into plaintext. Recall that a message is a vector of real numbers. Once we have a message vector we can use the `MakeCKKSPackedPlaintext` to encode the message into a plaintext.

In [ ]:
# create a message vector
msg_0 = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

ptxt_0 = context.MakeCKKSPackedPlaintext(msg_0)
ptxt_0

Before we can do anyhting more with the context we need to create a public and a private key. `KeyGen` returns a key pair instance which has a member `pubicKey` and `secretKey`.

In [ ]:
keys = context.KeyGen()

Using the `secretKey` we can encrypt the plaintext

In [ ]:
ctxt_0 = context.Encrypt(keys.publicKey, ptxt_0)

Let's see what happens if we decrypt the ciphertext




In [ ]:
# first print the message that is encrypted in the ciphertext
print(f'orginial message is: {msg_0}')
decryption = context.Decrypt(keys.secretKey,ctxt_0)
print('decrypted message is:')
decryption

Oh. This looks like a lot more than we were expecting. If you inspect the decryption there a (at least) two things you should notice.  

1.   If you look at the first few decrypted values and compare them to the message what do you notice? How do you explain your observation?
2.  Look at the number of items. How many values should it be? What do you notice about most of the values?

Think about your answers for a moment before you expand the cell below.

### Answers



1.  We can see that the encryption of `0` does not quite decrypt to `0` again but rather something very small. The absolute value is around $10^{-12}$ to $10^{-13}$ (the exact value differs as it depends on your exact scheme parameters). We don't get an exact decryption of `0` since CKKS is not an exact scheme and does incude some approximation error.

2. Our decryption contains `8192` values. If you recall from earlier that we observed a ring dimension $N$ of `16384`. CKKS plaintexts encode vectors of length $N/2$. In the decryption, we see that the vast majority (everything after our message) is nearly `0` not quite `0` for the same reason our encryption of `0` doesn't decrypt to exactly `0`. All these "0"s mean that we are wasting a lot of space in the ciphertexts. 8186 slots, to be precise. We'll discuss later how we can make better use of the slots.



## Ciphertext Operations

So far we have done nothing that we can't do with other public key schemes. Let's create a second ciphertext and see some operations in action.

In [ ]:
msg_1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
ptxt_1 = context.MakeCKKSPackedPlaintext(msg_1)
ctxt_1 = context.Encrypt(keys.publicKey, ptxt_1)

Like most other operations arithmetic operations are also accessible through the context.

In OpenFHE most operations that perfom an operation are prefixed with `Eval`. E.g. addition is `EvalAdd`.

We've seen before that currently plaintexts and ciphertexts contain many more values than we use. We can tell a plaintext how many values inside are useful to use by using the `SetLenght` methode.

In [ ]:
# add the two ciphertexts together
addition = context.EvalAdd(ctxt_0, ctxt_1)
decryption = context.Decrypt(keys.secretKey,addition)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
# let's compare that plain computation
print('plain result')
print([x + y for x,y in zip(msg_0,msg_1)])

In [ ]:
# We can also do subtraction
subtraction = context.EvalSub(addition, ctxt_1)
decryption = context.Decrypt(keys.secretKey,subtraction)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
# the plain results shoud be the original msg_0 or at least close to it
print('plain result')
print(msg_0)

Not only can we add ciphertexts to ciphertexts. We can also perform the operation with a ciphertext and plaintext. The plaintext can be encoded or unencoded. In the unencoded case, we use a scalar. During the operation, the scalar will be encoded into a plaintext where all slots contain the scalar.

Try it for youself below:

In [ ]:
# addition of a ciphertext and plaintext
print('ciphertext + plaintext')


addition =

decryption = context.Decrypt(keys.secretKey,addition)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
# let's compare that plain computation
print('plain result')
print([x + y for x,y in zip(msg_0,msg_0)])

In [ ]:
# addition of a ciphertext and a scalar
print('ciphertext + scalar')

scalar = # choose a scalar
addition = # add ctxt_0 and the scalar

decryption = context.Decrypt(keys.secretKey,addition)
# we are only interessted in the first 6 values
# decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
# let's compare that plain computation
print('plain result')
print([x + scalar for x in msg_0])

 In addition to addition we can also perform multiplication. OpenFHE has two multiplication methods. `EvalMultNoRelin` and `EvalMutl`. The first preforms multiplication without relineratization, the second relinearizes the ciphertext after multipication. Let's see some multiplication in action.

In [ ]:
multiplication = context.EvalMultNoRelin(ctxt_0, ctxt_1)
decryption = context.Decrypt(keys.secretKey,multiplication)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
print('plain result')
print([x * y for x,y in zip(msg_0, msg_1)])


Looks pretty good. Let's do some one more multiplication and square the result.

In [ ]:
multiplication = context.EvalMultNoRelin(multiplication, multiplication)
decryption = context.Decrypt(keys.secretKey, multiplication)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
print('plain result')
print([(x * y)*(x * y) for x,y in zip(msg_0, msg_1)])

That failed. We have run out of "prescion". Additionally, so far we have not reliniarized our ciphertext, so the number of polynomials just keeps growing. Let's solve that problem first. If we try to run multiplication with relinearization `EvalMult` or relinearization on its own `Relinearize` it will fail. You can see that in the next two cells.

In [ ]:
context.Relinearize(multiplication)

In [ ]:
multiplication = context.EvalMult(ctxt_0, ctxt_1)

We need to create relinearization keys first. OpenFHE calls them multiplication keys. We can create them using the context and the secret key by calling `EvalMultKeysGen`.

Let's think about what this means for a moment. If we want to send our encrypted data to someone for encrypted computation, they do not only need our data; we also need to provide them with relinearization keys.

In [ ]:
context.EvalMultKeysGen(keys.secretKey)

Now we should be able to relinearize our ciphertext from ealier, right?

In [ ]:
relined = context.Relinearize(multiplication)
decryption = context.Decrypt(keys.secretKey, relined)

That didn't work. Why do you think that is?

### Answer



When we multiply two ciphertext and don't relinearize they size grows from 2 to 3. If we perform further multiplications the size is grows even larger. By default, the relinearization keys are created to take a ciphertext from size 3 back down to 2. Unless you have a good reason you should always perform relinearization after a multiplication. Thankfully, OpenFHE does that for us.

##  Deeper and more Complext Computation

So far, all the computations we have done were pretty simple. Let's step it up a bit.

With what we learned so far, compute the polynomial $x^2+2x+1$ on the ciphertext `ctxt_0` and store result in `result`

In [ ]:
# x^2+ 2x + 1

# add your code here

result = #

decryption = context.Decrypt(keys.secretKey, result)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
print('plain result')
print([x**2 + 2*x + 1 for x in msg_0])


Nicely done, but what about  $3x^2+2x+1$? Copy your code from above into the cell below and modify it to compute $3x^2+2x+1$

In [ ]:
# 3x^2 + 2x + 1

sqr = context.EvalMult(ctxt_0, ctxt_0)
sqr = context.EvalMult(sqr,3)

two_x = context.EvalMult(ctxt_0,2)

result = context.EvalAdd(sqr,two_x)
result = context.EvalAdd(result,1)


decryption = context.Decrypt(keys.secretKey, result)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
print('plain result')
print([3*x**2 + 2*x + 1 for x in msg_0])


Once again it failed. We need to change the encryption parameters. To do that we need to create a new context. What do you think we need to change?

In [ ]:
parameters = openfhe.CCParamsCKKSRNS()

# change the parameters so that we can evaluate 3x^2 + 2x + 1

parameters.

context = openfhe.GenCryptoContext(parameters)
context.Enable(openfhe.PKESchemeFeature.PKE)
context.Enable(openfhe.PKESchemeFeature.KEYSWITCH)
context.Enable(openfhe.PKESchemeFeature.LEVELEDSHE)
context.Enable(openfhe.PKESchemeFeature.ADVANCEDSHE)

print('parameter')
print(str(parameters))
print('ring dimension')
print(context.GetRingDimension())

keys = context.KeyGen()
context.EvalMultKeysGen(keys.secretKey)
ptxt_0 = context.MakeCKKSPackedPlaintext(msg_0)
ctxt_0 = context.Encrypt(keys.publicKey, ptxt_0)

Let's see if that worked. OpenFHE provides a function `EvalPoly` to evaluate a polynomial on a ciphertext. We simply need to pass the coeffecients to the functions going from lowest power coeffiecent to highest. So in our case of $3x^2+2x+1$ we need to pass `[1,2,3]`. The degree of the polynomial will be automatically determined as as the lenght of the coefficie

In [ ]:
result = context.EvalPoly(ctxt_0, [1,2,3])

decryption = context.Decrypt(keys.secretKey, result)
# we are only interessted in the first 6 values
decryption.SetLength(6)
# get the decoded values
print('encrypted result')
print(decryption.GetRealPackedValue())
print('plain result')
print([3*x**2 + 2*x + 1 for x in msg_0])

## A "real-world" Example

Now that we have seen the fundamental building blocks, it is time to put them together. Our goal is to recognize handwritten digits. Yes, the (in)famous MNIST handwritten digit dataset. If you haven't heard of it or want to see some examples, have a look at the Wikipedia page (https://en.wikipedia.org/wiki/MNIST_database).

We'll use the dataset included in scikit-learn. Here, every digit is a 8x8 pixel greyscale image. First, let's tackle a simple task. Distinguish `0`s and `1`s.

First, we load the data and extract all the `0`s and `1`s.

In [ ]:
import sklearn
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt


digits = datasets.load_digits()

zeroes = digits['images'][digits['target'] == 0]
ones = digits['images'][digits['target'] == 1]

Let's take a closer look at the data

In [ ]:
zeroes.shape, ones.shape

In [ ]:
plt.imshow(zeroes[0])
plt.show()
plt.imshow(ones[0])

Next, we combine all our `0`s and `1`s, and create appropiate labels. Then we'll shuffle the data.

In [ ]:
x = np.concatenate((zeroes, ones))
y = np.concatenate((np.zeros(len(zeroes)), np.ones(len(ones))))
x,y = sklearn.utils.shuffle(x,y)
x,y, y.shape

Our new, reduced dataset contains 360 instances. Let's set 60 instances aside for testing and use the rest to train a classifier.

In [ ]:
x_train, y_train = x[:300], y[:300]
x_test, y_test = x[300:], y[300:]

We'll use the `Perceptron` classifier. The perceptron is a very simple classifier. It learns a linear decision boundary between two classes. We won't go into how the learning works; we are only interested in making the decisions. It learns a weight $w_i$ for each feature $x_i$ and bias $b$.

In our case an instance is an image of a hand-written digit. It consists of 64 features, which are the pixel values. To work with the perceptron we flatten the 2D images into a 1D vector.

To make a decision the perceptron takes a vector $x$ and the learned weight vector $w$ and computes the dot product $x \cdot w$ and then adds the bias $b$ to the result. If the result is greater than 0, we have class 1 otherwise class 0.

Let's train the perceptron on our training data and see how well it performs on the unseen test data.

In [ ]:
from sklearn.linear_model import Perceptron

classifier = Perceptron()
classifier.fit(x_train.reshape(len(x_train),-1), y_train)
classifier.score(x_test.reshape(len(x_test),-1), y_test)

Fantastic. We learned the perfect classifier. Let's see what we learned. In sklearn the weights are stored in the `coef_` and bias in the `intercept_` members of the classifier.

In [ ]:
classifier.coef_, classifier.intercept_

Now we have everything we need to run the perceptron on encrypted data. Below you'll find a function stub. The function expects the weights (`coef_`) and bias (`intercept_`). Additionally, you need to pass a list of ciphertexts that encrypt an instance, and finally the context. The function than returns a list of ciphertexts (although this list might contain only a single ciphertext, depending on your implementation)

At this point you also need to think about your encoding/encyrption. Here are some things you need to think of:


*   How many ciphertexts do you need?
*   Which slot will a given feature go into?
*   How does that influence the implemtation of the dot product?

There are two function stubs that encrypt and instance into a list of ciphertexts and decrypt the result of your pereceptron.

In [ ]:
from typing import List

def encrypted_perceptron(coef:np.ndarray,
                         intercept:np.ndarray, x: List[openfhe.Ciphertext],
                         context: openfhe.CryptoContext) -> List[openfhe.Ciphertext]:
  """
  Compute the perceptron classification on encrypted data.
  The formula is: x * coef + intercept, where is * the dot
  product.

  Arguments:
  coef: np.ndarray the percepetron coefficients/weights
  intercept: np.ndarray the perceptron intercept/bias
  x: list of openfhe.Ciphertext the encrypted input data
  context: openfhe.Context

  Returns:
  list of openfhe.Ciphertext the encrypted result
  """
  # if you want the weights to be a simple 1D vector and the bias just a scalar
  # uncomment the next to lines. Otherwise they will be 2D matrix and 1D vector instead
  # coef = coef.reshape(-1)
  # intercept = intercept[0]
  pass


def encrypt_instance(x: np.ndarray, context: openfhe.CryptoContext,
                     pub_key: openfhe.PublicKey) -> List[openfhe.Ciphertext]:
  """
  Encrypt an instance and return a list of ciphertexts
  """
  pass


def decrypt_result(encryption: List[openfhe.Ciphertext],
                   context: openfhe.CryptoContext,
                   sec_key: openfhe.PrivateKey) -> np.ndarray:
  """
  Decrypt a list of ciphertexts and return a classification result as a numpy array
  """
  pass

You can use the cell below to test your implementation. It also provides some timing information on how long your functions take.

In [ ]:
# select an instance
instance_no = 0
x = x_test[instance_no]
y = y_test[instance_no]

# get the plain data
print('plain computation')
%time plain_result = classifier.decision_function(x.reshape(1,-1))
print(plain_result)


# encrypt the instance
print('encrypting data')
%time encryption = encrypt_instance(x, context, keys.publicKey)

print('encrypted computation')
%time encrypted_result = encrypted_perceptron(classifier.coef_, classifier.intercept_, encryption, context)
# decrypt and decode the result here
decrypted_result = decrypt_result(encrypted_result, context, keys.secretKey)
print(decrypted_result)


print(f'plain result:{plain_result}, encyrpted result {decrypted_result}')



The issue with this implementation is that we are leaving many of the ciphertext slots unfilled. Can you modify your implementaion to so it can process the entire test data at once?

Put your batched implementation into the cell below

In [ ]:
def encrypted_perceptron_batch(coef:np.ndarray,
                         intercept:np.ndarray, x: List[openfhe.Ciphertext],
                         context: openfhe.CryptoContext) -> List[openfhe.Ciphertext]:
  """
  Compute the perceptron classification on encrypted data.
  The formula is: x * coef + intercept, where is * the dot
  product.

  Arguments:
  coef: np.ndarray the percepetron coefficients/weights
  intercept: np.ndarray the perceptron intercept/bias
  x: list of openfhe.Ciphertext the encrypted input data
  context: openfhe.Context

  Returns:
  list of openfhe.Ciphertext the encrypted result
  """
  # if you want the weights to be a simple 1D vector and the bias just a scalar
  # uncomment the next to lines. Otherwise they will be 2D matrix and 1D vector instead
  # coef = coef.reshape(-1)
  # intercept = intercept[0]
  pass

def encrypt_batch(x: np.ndarray, context: openfhe.CryptoContext,
                     pub_key: openfhe.PublicKey) -> List[openfhe.Ciphertext]:
  """
  Encrypt a batch of instance and return a list of ciphertexts
  """
   pass

def decrypt_batch(encryption: List[openfhe.Ciphertext],
                   context: openfhe.CryptoContext,
                   sec_key: openfhe.PrivateKey, batchsize: int) -> np.ndarray:
  """
  Decrypt a list of ciphertexts and return a classification result as a numpy array
  batchsize is the nubmer of results that we want to return
  """
  pass

For convience let's turn all instances into flat vectors

In [ ]:
x_flat = x_test.reshape(x_test.shape[0], -1)

Below is the evaluation code with timing information for the batched computation. Do you notice anything about the computation time?

In [ ]:
# get the plain data
print('plain computation')
%time plain_result = classifier.decision_function(x_flat)
print(plain_result)

# batch encrypt the istances
print('batch encrypting')
%time encryption = encrypt_batch(x_flat, context, keys.publicKey)

print('encrypted batch computation')
%time encrypted_result = encrypted_perceptron_batch(classifier.coef_, classifier.intercept_, encryption, context)
# decrypt and decode the result here
decrypted_result = decrypt_batch(encrypted_result, context, keys.secretKey, x_flat.shape[0])
print(decrypted_result)

# is the comutation reasonably close?
print(f'mean squared error between plain and encyrpted computation ? {np.mean((plain_result- decrypted_result)**2)}')

Great! But while we were implementing the code, the requiments of our project changed. The new requirment is that the output of our programm should be the probablity that the instance was a `1`. Let's think about this for a second. Large positive values mean, we are very sure that the instance is `1`. "Large" (absolute value) negative values mean are very sure the result is `0` or not a `1`. If the output is 0 it is 50/50 between `0` and `1` menaing we are maximally unsure.

Luckilly we can simply apply the sigmoid function to the result and that should give us exactly what we need.

Let's first define the sigmoid function and run it over the plain results.

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

sigmoid(plain_result)

The computation is overflowing on plain data already. This will not be good on encrypted data. Let's scale the data by a  factor of 100.

In [ ]:
plain_result_scaled = plain_result / 100
sigmoid(plain_result_scaled)

Looks pretty good. The problem is that we can't easily run thins on encrypted data. Let's approximate the function using a polynomial. First we select the interval that we want to approximate over. We'll use the minimum and maximum in our data and then plot what the sigmoid function looks over that interval.

In [ ]:
x_values = np.arange(plain_result_scaled.min(), plain_result_scaled.max(), 0.1)
y_values = sigmoid(x_values)
plt.plot(x_values, y_values)

Next, we can use those values to approximate the function. Try a few different degrees below. If you run into problems with noise and decryption errors you can uncomment the cell below and change the crypto parameters.

Experiment with different combinations of poynomial degree, mutliplicaive and scaling mod size.

In [ ]:
# parameters = openfhe.CCParamsCKKSRNS()

# context = openfhe.GenCryptoContext(parameters)
# context.Enable(openfhe.PKESchemeFeature.PKE)
# context.Enable(openfhe.PKESchemeFeature.KEYSWITCH)
# context.Enable(openfhe.PKESchemeFeature.LEVELEDSHE)
# context.Enable(openfhe.PKESchemeFeature.ADVANCEDSHE)

# keys = context.KeyGen()
# context.EvalMultKeysGen(keys.secretKey)


# # batch encrypt the istances
# print('batch encrypting')
# %time encryption = encrypt_batch(x_flat, context, keys.publicKey)

# print('encrypted batch computation')
# %time encrypted_result = encrypted_perceptron_batch(classifier.coef_, classifier.intercept_, encryption, context)

In [ ]:
from numpy.polynomial.polynomial import Polynomial

degree = 1

poly_coeff = np.polyfit(x_values, y_values, degree)
poly = Polynomial(poly_coeff[::-1])
poly

Now that we have a polynomial, let's plot that and compare it to the sigmoid function. Are you happy with it?

In [ ]:
plt.plot(x_values, poly(x_values))
plt.plot(x_values, y_values)


Remeber that we scaled the plain results? We need to do the same thing to the encrypted result.

In [ ]:
encrypted_scaled = context.EvalMult(encrypted_result[0], 1/100)

Now we can compute the polynoial on encyrpted data

In [ ]:
%time sigmoid_enc = context.EvalPoly(encrypted_scaled, poly.coef.tolist())

Time to the decrypt the data

In [ ]:
sigmoid_decrypted = decrypt_batch([sigmoid_enc], context, keys.secretKey, plain_result.shape[0])
sigmoid_decrypted

Compute the mean squared error (mse) of the polynomial on plain data and encrypted data. This should be very small

In [ ]:
np.mean((poly(plain_result_scaled) - np.array(sigmoid_decrypted))**2)

But we actually don't want to compute the polynomial. We want to compute the sigmoid function. Compute the MSE of the sigmoid function on the plain scaled results and the polynomial on encrypted data. If the MSE is too large try a different approximation degree and/or crypto parameters?

Bonus question: If we assume everything large 0.5 is a `1` do any of the classifications change from plain data with the sigmoid function compared to encyrpted data with the approximation? If so how many?

In [ ]:
# you code goes here